In [75]:
# function definitions are in the script, no definitions in this ipynb allowed!
include("PES_models_bonding_features.jl") # need to reload everytime a change occurs

multirestart (generic function with 1 method)

In [2]:
# test using actual data:
H_data = readdlm("data/h3/h3_data.txt")
# load atomic coordinates:
H_coord = npzread("data/h3/h3_coord.npy")
R = H_data[:,1:end-1]; V = H_data[:, end]
n_data = size(R)[1]

6032

In [3]:
idxes = shuffleobs(1:n_data); # shuffle indexes

In [4]:
# split:
id_train, id_test = splitobs(idxes, at=0.8)
R_train = R[id_train,:]; V_train = V[id_train];
R_test = R[id_test,:]; V_test = V[id_test]
H_coord_train = H_coord[id_train,:,:]; H_coord_test = H_coord[id_test,:,:];

In [76]:
const_r_xy = 1.4172946 # H2
#const_r_xy = 1.9369 #OH+
N = 5 # hyperparam
ρ = f_ρ(R, const_r_xy)
q = f_q_bump(N, ρ)
i = f_i(q) # should be transformed to unique integers
ϵ = f_ϵ(i, q)
# for h:
α = f_α(ϵ)
β = f_β(ϵ)
#ub = 1.; lb = -1; θ = rand(2*N+2) .* (ub-lb) .+ lb; # +2 since it starts from 0th index
θ = ones(2*N+2);

In [77]:
n_data, n_d = size(R)
u = similar(R)
w = similar(u)
h = zeros(n_data, n_d, N+1) #stores the first sub primitive
compute_u_w_h!(u, w, h, θ, q, α, β, i, N)
#@benchmark compute_u_w_h!($u, $w, $h, $θ, $q, $α, $β, $i, $N)

In [78]:
x = similar(h)
y = similar(x)
compute_hw_gw!(x, y, w, h, q, N)
#@benchmark compute_hw_gw!($x, $y, $w, $h, $q, $N)

In [82]:
ρ,u,w,x,y = BUMP_feature(θ, R, const_r_xy, N);
@benchmark BUMP_feature($θ, $R, $const_r_xy, $N)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  276.500 μs …   6.522 ms  ┊ GC (min … max):  0.00% … 86.29%
 Time  (median):     333.700 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   420.378 μs ± 356.478 μs  ┊ GC (mean ± σ):  18.21% ± 17.43%

  ▃██▅▃▂▁                                                 ▂▂    ▂
  ███████▇▆▆▄▁▄▃▁▁▁▁▃▃▆▇▇▅▄▁▅▄▁▄▃▃▁▃▅▃▃▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅█████▇ █
  276 μs        Histogram: log(frequency) by time       1.96 ms <

 Memory estimate: 3.59 MiB, allocs estimate: 22.

In [84]:
e_pow = 3 #hyperparam, 3 seems to be the best
v_BUMP_di(u, w, ρ, e_pow)

6032×3 Matrix{Float64}:
 -0.116291    -0.0589616    -0.0404142
 -0.116291    -0.00785096   -0.00676472
 -0.10485     -0.10485      -0.0735249
 -0.10485     -0.10485      -0.0459331
 -0.10485     -0.0589616    -0.0444205
 -0.10485     -0.0589616    -0.0162825
 -0.10485     -0.00785096    0.0114707
 -0.10485      0.0305181     0.033634
 -0.0589616   -0.0589616    -0.0486053
 -0.0589616   -0.0589616    -0.0185286
 -0.0589616   -0.0589616     0.00757083
 -0.0589616   -0.00785096    0.00685003
 -0.0589616   -0.00785096    0.0282934
  ⋮                         
 -0.00524119   0.000996644  -0.000657669
  0.00499096   0.00329408   -2.0295e-5
  0.00219798   0.000851275   0.000800969
  0.00217524   0.00106892    0.00104344
  0.166349    -0.000956516  -0.000938756
  0.178884    -0.000874135  -0.000787083
  0.148848    -0.000952415  -0.000918866
  0.170505    -0.000753646  -0.00073287
  0.182865    -0.000905035  -0.000890453
  0.188465    -0.000863792  -0.000684584
  0.16392     -0.000866879  -0.0